In [1]:
# Load the necessary packages
import torch
import torchvision.transforms as transforms

from speckcn2.io import load_config
from speckcn2.preprocess import prepare_data, normalize_tags, train_test_split
from speckcn2.mlmodels import setup_model, setup_loss, setup_optimizer
from speckcn2.mlops import train, score
from speckcn2.postprocess import tags_distribution

In [2]:
# Load the configuration file
#config = load_config(sys.argv[1])
config = load_config('configuration.yaml')

In [3]:
# Get some parameters from YAML file
datadirectory = config['speckle']['datadirectory']

In [4]:
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using {device}.')

Using cpu.


In [5]:
# Define the transformation to apply to each image
transform = transforms.Compose([
    # Randomly rotate the image, since it is symmetric
    transforms.RandomRotation(degrees=(-180, 180)),
    # Take only the center of the image
    transforms.CenterCrop(config['speckle']['centercrop']),
    # Optionally, downscale it
    transforms.Resize(config['speckle']['resize']),
    transforms.ToTensor(),
])

In [6]:
# Load or preprocess the data
all_images, all_tags = prepare_data(datadirectory,
                                    transform,
                                    nimg_print=5,
                                    nreps=2)

*** Loading preprocessed data


In [7]:
# Normalize the tags between 0 and 1
dataset, normalize_tag, recover_tag = normalize_tags(all_images, all_tags)

In [8]:
# Split the data in training and testing
train_loader, test_loader = train_test_split(dataset,
                                             config['hyppar']['batch_size'],
                                             0.8)

*** There are 200 images in the dataset, 160 for training and 40 for testing.


In [9]:
# Load the model that you want to use
model, last_model_state = setup_model(config)
# and set the model to run on the device
model = model.to(device)

/home/simone/.virtualenvs/MALES/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/simone/.virtualenvs/MALES/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model at epoch 6


In [10]:
# Define the loss function and optimizer
criterion = setup_loss(config)
optimizer = setup_optimizer(config, model)

In [11]:
# Train the model...
model, average_loss = train(model, last_model_state, config, train_loader,
                            device, optimizer, criterion)
print(f'Finished Training, Loss: {average_loss:.4f}')

Training the model from epoch 6 to 6
Finished Training, Loss: 0.0000


In [12]:
# Now test the model
test_tags = score(model, test_loader, device, criterion, recover_tag,
                  datadirectory)

Item 0 loss: 0.0569
Item 1 loss: 0.3586
Item 2 loss: 0.0198
Item 3 loss: 0.1168
Item 4 loss: 0.1694
Item 5 loss: 0.0540
Item 6 loss: 0.0229
Item 7 loss: 0.1148
Item 8 loss: 0.0193
Item 9 loss: 0.0175
Item 10 loss: 0.1695
Item 11 loss: 0.0428
Item 12 loss: 0.0387
Item 13 loss: 0.0306
Item 14 loss: 0.3459
Item 15 loss: 0.3643
Item 16 loss: 0.0600
Item 17 loss: 0.0381
Item 18 loss: 0.0175
Item 19 loss: 0.0179
Item 20 loss: 0.0327
Item 21 loss: 0.0459
Item 22 loss: 0.0375
Item 23 loss: 0.0219
Item 24 loss: 0.1043
Item 25 loss: 0.0229
Item 26 loss: 0.1148
Item 27 loss: 0.3428
Item 28 loss: 0.1683
Item 29 loss: 0.0510
Item 30 loss: 0.0466
Item 31 loss: 0.1631
Item 32 loss: 0.0439
Item 33 loss: 0.0192
Item 34 loss: 0.0320
Item 35 loss: 0.1135
Item 36 loss: 0.1735
Item 37 loss: 0.1161
Item 38 loss: 0.0452
Item 39 loss: 0.0353


In [13]:
# Print some statistics of the screen tags
tags_distribution(dataset, test_tags, device, datadirectory)

Data shape: (200, 8)
Prediction shape: (40, 8)
Train mean: 0.3823397755622864
Train std: 0.28305110335350037
Prediction mean: 0.3716089725494385
Prediction std: 0.29408857226371765
